# Table Management

Apache Spark&trade; and Azure Databricks&reg; allow you to access and optimize data in managed and unmanaged tables.

-sandbox
### Optimization of Data Storage with Managed and Unmanaged Tables

A **managed table** is a table that manages both the data itself as well as the metadata.  In this case, a `DROP TABLE` command removes both the metadata for the table as well as the data itself.  

**Unmanaged tables** manage the metadata from a table such as the schema and data location, but the data itself sits in a different location, often backed by a blob store like the Azure Blob Storage. Dropping an unmanaged table drops only the metadata associated with the table while the data itself remains in place.

<div><img src="https://files.training.databricks.com/images/eLearning/ETL-Part-2/managed-and-unmanaged-tables.png" style="height: 400px; margin: 20px"/></div>

### Writing to a Managed Table

Managed tables allow access to data using the Spark SQL API.

Run the cell below to mount the data.

In [5]:
%run "./Includes/Classroom-Setup"

Create a DataFrame.

In [7]:
df = spark.range(1, 100)

display(df)

id
1
2
3
4
5
6
7
8
9
10


Register the table.

In [9]:
df.write.mode("OVERWRITE").saveAsTable("myTableManaged")

Use `DESCRIBE EXTENDED` to describe the contents of the table.  Scroll down to see the table `Type`.
Notice the location is also `dbfs:/user/hive/warehouse/mytable`.

In [11]:
%sql
DESCRIBE EXTENDED myTableManaged

col_name,data_type,comment
id,bigint,null
,,
# Detailed Table Information,,
Database,default,
Table,mytablemanaged,
Owner,root,
Created Time,Thu Jul 25 03:33:16 UTC 2019,
Last Access,Thu Jan 01 00:00:00 UTC 1970,
Created By,Spark 2.4.3,
Type,MANAGED,


In [12]:
%fs ls 	dbfs:/user/hive/warehouse/mytablemanaged

path,name,size
dbfs:/user/hive/warehouse/mytablemanaged/_SUCCESS,_SUCCESS,0
dbfs:/user/hive/warehouse/mytablemanaged/_committed_1296440421623173483,_committed_1296440421623173483,416
dbfs:/user/hive/warehouse/mytablemanaged/_started_1296440421623173483,_started_1296440421623173483,0
dbfs:/user/hive/warehouse/mytablemanaged/part-00000-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-4-1-c000.snappy.parquet,part-00000-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-4-1-c000.snappy.parquet,538
dbfs:/user/hive/warehouse/mytablemanaged/part-00001-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-5-1-c000.snappy.parquet,part-00001-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-5-1-c000.snappy.parquet,542
dbfs:/user/hive/warehouse/mytablemanaged/part-00002-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-6-1-c000.snappy.parquet,part-00002-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-6-1-c000.snappy.parquet,542
dbfs:/user/hive/warehouse/mytablemanaged/part-00003-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-7-1-c000.snappy.parquet,part-00003-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-7-1-c000.snappy.parquet,542


### Writing to an Unmanaged Table

Write to an unmanaged table by adding an `.option()` that includes a path.

In [14]:
df.write.mode("OVERWRITE").option('path', '/tmp/myTableUnmanaged').saveAsTable("myTableUnmanaged")

-sandbox
Now examine the table type and location of the data.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> An external table is the same as an unmanaged table.

In [16]:
%sql
DESCRIBE EXTENDED myTableUnmanaged

col_name,data_type,comment
id,bigint,null
,,
# Detailed Table Information,,
Database,default,
Table,mytableunmanaged,
Owner,root,
Created Time,Thu Jul 25 03:35:57 UTC 2019,
Last Access,Thu Jan 01 00:00:00 UTC 1970,
Created By,Spark 2.4.3,
Type,EXTERNAL,


### Dropping Tables

Take a look at how dropping tables operates differently in the two cases below.

Look at the files backing up the managed table.

In [19]:
%fs ls dbfs:/user/hive/warehouse/mytablemanaged

path,name,size
dbfs:/user/hive/warehouse/mytablemanaged/_SUCCESS,_SUCCESS,0
dbfs:/user/hive/warehouse/mytablemanaged/_committed_1296440421623173483,_committed_1296440421623173483,416
dbfs:/user/hive/warehouse/mytablemanaged/_started_1296440421623173483,_started_1296440421623173483,0
dbfs:/user/hive/warehouse/mytablemanaged/part-00000-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-4-1-c000.snappy.parquet,part-00000-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-4-1-c000.snappy.parquet,538
dbfs:/user/hive/warehouse/mytablemanaged/part-00001-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-5-1-c000.snappy.parquet,part-00001-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-5-1-c000.snappy.parquet,542
dbfs:/user/hive/warehouse/mytablemanaged/part-00002-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-6-1-c000.snappy.parquet,part-00002-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-6-1-c000.snappy.parquet,542
dbfs:/user/hive/warehouse/mytablemanaged/part-00003-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-7-1-c000.snappy.parquet,part-00003-tid-1296440421623173483-b8926f04-48b8-45e9-bbdd-a904b3c0d480-7-1-c000.snappy.parquet,542


Drop the table.

In [21]:
%sql
DROP TABLE myTableManaged

Next look at the underlying data.

In [23]:
%fs ls dbfs:/user/hive/warehouse/mytablemanaged

at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.listStatus(NativeAzureFileSystem.java:2316)
	at com.databricks.backend.daemon.data.client.DBFSV2$$anonfun$listStatus$1$$anonfun$apply$2.apply(DatabricksFileSystemV2.scala:95)
	at com.databricks.backend.daemon.data.client.DBFSV2$$anonfun$listStatus$1$$anonfun$apply$2.apply(DatabricksFileSystemV2.scala:92)
	at com.databricks.s3a.S3AExeceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:119)
	at com.databricks.backend.daemon.data.client.DBFSV2$$anonfun$listStatus$1.apply(DatabricksFileSystemV2.scala:92)
	at com.databricks.backend.daemon.data.client.DBFSV2$$anonfun$listStatus$1.apply(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:369)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:474)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:271)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:474)
	at com.databricks.logging.UsageLogging$class.recordOperation(UsageLogging.scala:350)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:474)
	at com.databricks.backend.daemon.data.client.DBFSV2.listStatus(DatabricksFileSystemV2.scala:91)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:146)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$ls$1.apply(DBUtilsCore.scala:86)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$ls$1.apply(DBUtilsCore.scala:85)
	at com.databricks.backend.daemon.dbutils.FSUtils$.com$databricks$backend$daemon$dbutils$FSUtils$$withFsSafetyCheck(DBUtilsCore.scala:81)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:85)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.ls(DbfsUtilsImpl.scala:34)
	at line720e9a2d1a574508b76b395757d6efe039.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3936276780925018:1)
	at line720e9a2d1a574508b76b395757d6efe039.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-3936276780925018:44)
	at line720e9a2d1a574508b76b395757d6efe039.$read$$iw$$iw$$iw$$iw.<init>(command-3936276780925018:46)
	at line720e9a2d1a574508b76b395757d6efe039.$read$$iw$$iw$$iw.<init>(command-3936276780925018:48)
	at line720e9a2d1a574508b76b395757d6efe039.$read$$iw$$iw.<init>(command-3936276780925018:50)
	at line720e9a2d1a574508b76b395757d6efe039.$read$$iw.<init>(command-3936276780925018:52)
	at line720e9a2d1a574508b76b395757d6efe039.$read.<init>(command-3936276780925018:54)
	at line720e9a2d1a574508b76b395757d6efe039.$read$.<init>(command-3936276780925018:58)
	at line720e9a2d1a574508b76b395757d6efe039.$read$.<clinit>(command-3936276780925018)
	at line720e9a2d1a574508b76b395757d6efe039.$eval$.$print$lzycompute(<notebook>:7)
	at line720e9a2d1a574508b76b395757d6efe039.$eval$.$print(<notebook>:6)
	at line720e9a2d1a574508b76b395757d6efe039.$eval.$print(<notebook>)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:793)
	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1054)
	at scala.tools.nsc.interpreter.IMain$WrappedRequest$$anonfun$loadAndRunReq$1.apply(IMain.scala:645)
	at scala.tools.nsc.interpreter.IMain$WrappedRequest$$anonfun$loadAndRunReq$1.apply(IMai

The data was deleted so spark will not find the underlying data. Perform the same operation with the unmanaged table.

In [25]:
%fs ls /tmp/myTableUnmanaged

path,name,size
dbfs:/tmp/myTableUnmanaged/_SUCCESS,_SUCCESS,0
dbfs:/tmp/myTableUnmanaged/_committed_3869042010224724826,_committed_3869042010224724826,418
dbfs:/tmp/myTableUnmanaged/_started_3869042010224724826,_started_3869042010224724826,0
dbfs:/tmp/myTableUnmanaged/part-00000-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-8-1-c000.snappy.parquet,part-00000-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-8-1-c000.snappy.parquet,538
dbfs:/tmp/myTableUnmanaged/part-00001-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-9-1-c000.snappy.parquet,part-00001-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-9-1-c000.snappy.parquet,542
dbfs:/tmp/myTableUnmanaged/part-00002-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-10-1-c000.snappy.parquet,part-00002-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-10-1-c000.snappy.parquet,542
dbfs:/tmp/myTableUnmanaged/part-00003-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-11-1-c000.snappy.parquet,part-00003-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-11-1-c000.snappy.parquet,542


Drop the unmanaged table.

In [27]:
%sql
DROP TABLE myTableUnmanaged

See if the data is still there.

In [29]:
%fs ls /tmp/myTableUnmanaged

path,name,size
dbfs:/tmp/myTableUnmanaged/_SUCCESS,_SUCCESS,0
dbfs:/tmp/myTableUnmanaged/_committed_3869042010224724826,_committed_3869042010224724826,418
dbfs:/tmp/myTableUnmanaged/_started_3869042010224724826,_started_3869042010224724826,0
dbfs:/tmp/myTableUnmanaged/part-00000-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-8-1-c000.snappy.parquet,part-00000-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-8-1-c000.snappy.parquet,538
dbfs:/tmp/myTableUnmanaged/part-00001-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-9-1-c000.snappy.parquet,part-00001-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-9-1-c000.snappy.parquet,542
dbfs:/tmp/myTableUnmanaged/part-00002-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-10-1-c000.snappy.parquet,part-00002-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-10-1-c000.snappy.parquet,542
dbfs:/tmp/myTableUnmanaged/part-00003-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-11-1-c000.snappy.parquet,part-00003-tid-3869042010224724826-2d7fca3f-6728-4115-8468-24f3a7017a44-11-1-c000.snappy.parquet,542


## Review
**Question:** What happens to the original data when I delete a managed table?  What about an unmanaged table?  
**Answer:** Deleting a managed table deletes both the metadata and the data itself. Deleting an unmanaged table does not delete the original data.

**Question:** What is a metastore?  
**Answer:** A metastore is a repository of metadata such as the location of where data is and the schema information. A metastore does not include the data itself.

## Additional Topics & Resources

**Q:** Where can I find out more about connnecting to my own metastore?  
**A:** Take a look at the <a href="https://docs.azuredatabricks.net/user-guide/advanced/external-hive-metastore.html" target="_blank">Databricks documentation for more details</a>

**Extra Practice:** Apply what you learned in this module by completing the optional [Custom Transformations, Aggregating, and Loading]($./Optional/Custom-Transformations) exercise.